# Prepare Ames Housing Data

Contents
 - start
  - packages
  - directories and paths
 - data manipulation
   - dealing with categorical variables
   - train - val - test split
 - save data

Sources:
http://ww2.amstat.org/publications/jse/v19n3/decock.pdf

Copyright (C) 2018 Alan Chalk  
Please do not distribute or publish without permission.

## Start_.

**Import any packages needed** 

In [26]:
import os
import re

import numpy as np
import pandas as pd
import pickle

#from sklearn.preprocessing import OneHotEncoder

from matplotlib import pyplot as plt
%matplotlib inline

**Set directories and paths**

In [27]:
# Set directories
print(os.getcwd())
dirRawData = "../input/"
dirPData   = "../PData/"
dirPOutput = "../POutput/"

/home/jovyan/Projects/AmesHousing/PCode


**Load data**

In [28]:
#store = pd.HDFStore(dirPData + '02_df_all.h5')
#df_all = pd.read_hdf(store, 'df_all')
#store.close()
f_name = dirPData + '02_df.pickle'

with (open(f_name, "rb")) as f:
    dict_ = pickle.load(f)

df_all = dict_['df_all']

del f_name, dict_

In [29]:
f_name = dirPData + '02_vars.pickle'

with open(f_name, "rb") as f:
    dict_ = pickle.load(f)

vars_ind_numeric = dict_['vars_ind_numeric']
vars_notToUse = dict_['vars_notToUse']
var_dep = dict_['var_dep']

dict_.keys()
del f_name, dict_

### Data Manipulation

**Deal with categorical variables**

For h2o (and lightgbm) we do not need to onehot these variables - just flag them as categorical / factors

TO DO

 - Create vars_ind_categorical, a list of the categorical (dtypes == 'object') variables
 - Note: one was is along the following lines 
  - df_all.dtypes gives a pandas series containing the dtypes (try it)
  - df_all.dtypes == 'object' gives a series of True / False (try it)
  - df_all.columns gives the pandas Index of column names
  - the above index can be sliced with the True / False of dtypes and then the method .tolist() can be applied

In [30]:
# df_all.dtypes
# df_all.dtypes == 'object'
vars_ind_categorical = [var for var in df_all.columns if df_all[var].dtype == 'object']
vars_ind_categorical
# df_all.head(10)

['ms_subclass',
 'ms_zoning',
 'street',
 'alley',
 'lot_shape',
 'land_contour',
 'lot_config',
 'land_slope',
 'neighborhood',
 'condition_1',
 'condition_2',
 'bldg_type',
 'house_style',
 'roof_style',
 'roof_matl',
 'exterior_1st',
 'exterior_2nd',
 'mas_vnr_type',
 'exter_qual',
 'exter_cond',
 'foundation',
 'bsmt_qual',
 'bsmt_cond',
 'bsmt_exposure',
 'bsmtfin_type_1',
 'bsmtfin_type_2',
 'heating',
 'heating_qc',
 'central_air',
 'electrical',
 'kitchen_qual',
 'functional',
 'fireplace_qu',
 'garage_type',
 'garage_finish',
 'garage_qual',
 'garage_cond',
 'paved_drive',
 'fence',
 'misc_feature',
 'sale_type',
 'sale_condition']

TODO

 - What is the cardinality of the categorical variables?  

In [33]:
df_all[vars_ind_categorical].nunique() #show cardinality of each categorical variable

ms_subclass       14
ms_zoning          5
street             2
alley              3
lot_shape          4
land_contour       4
lot_config         5
land_slope         3
neighborhood      24
condition_1        7
condition_2        2
bldg_type          5
house_style        7
roof_style         5
roof_matl          2
exterior_1st      10
exterior_2nd      12
mas_vnr_type       4
exter_qual         4
exter_cond         4
foundation         5
bsmt_qual          5
bsmt_cond          4
bsmt_exposure      5
bsmtfin_type_1     7
bsmtfin_type_2     7
heating            2
heating_qc         4
central_air        2
electrical         3
kitchen_qual       4
functional         5
fireplace_qu       6
garage_type        7
garage_finish      4
garage_qual        4
garage_cond        5
paved_drive        3
fence              5
misc_feature       2
sale_type          5
sale_condition     6
dtype: int64

We can see below it is not too high - therefore we will simply one-hot all the categorical variables.

### Example of one-hot

In [42]:
# What are the various roof styles?
print(df_all['roof_style'].unique())
print(len(df_all['roof_style'].unique()))

['Hip' 'Gable' 'Mansard' 'Gambrel' 'Flat']
5


In [35]:
df_all.loc[15:20, 'roof_style']
# df_all['roof_style'].value_counts() #check different types and freq of roof styles

15      Hip
16    Gable
17      Hip
18    Gable
19    Gable
20      Hip
Name: roof_style, dtype: object

**onehot encoding using pandas**

TODO

 - Run the code below
 - What is the difference between using drop_first = False and = True?
 <b>ANS: drop_first = True drops the first roof style</b>

In [36]:
pd.get_dummies(df_all['roof_style'], drop_first = False)[15:20]

,Flat,Gable,Gambrel,Hip,Mansard
15,0,0,0,1,0
16,0,1,0,0,0
17,0,0,0,1,0
18,0,1,0,0,0
19,0,1,0,0,0


In [37]:
pd.get_dummies(df_all['roof_style'], drop_first = True)[15:20]

,Gable,Gambrel,Hip,Mansard
15,0,0,1,0
16,1,0,0,0
17,0,0,1,0
18,1,0,0,0
19,1,0,0,0


### Discussion on ill-defined models

 - If we one-hot every level of a categorical variable, is the linear regression model well defined?
 <b>no</b>
 - Should we drop_first when creating dummies?  <b>yes,we have to</b>
 - What is the impact on linear regression if we do?  
 - What is the impact on tree based methods? <b> don't </b>

### onehot encoding using sklearn

Within recent versions of sklearn (>=0.20), it is possible to transform categorical variables very simply, using the standard instantiate, fit and tranform procedure we have used before.  It would go something like this:

 - instantiate a OneHotEncoder(), call it enc_
 - fit it to df_all[['roof_style']]
 - use its  .transform method to transform df_all.loc[0:20, ['roof_style']]

Currently, the container we are using has sklearn version 0.19 in which it is not so simple to transform text variables.  Therefore we will use Pandas to do the onehot encoding.  Let's see how this might work



In [128]:
# Create a copy of df_all and call it df_all_onehot
# df_all_onehot = df_all.copy()

# Choose a column name
# col = 'ms_subclass'

# Create dummies for that column using pd.get_dummies with drop_first=False
# df_oh = pd.get_dummies(df_all[col], drop_first=False)
# print(df_oh)
# Find the sum down each column - which is the most frequent column?
# print(df_oh.sum(axis = 0))

# Find the column name of the most frequent column
# col_mostFreq = df_oh.sum(axis = 0).idxmax()

# Drop the column of the most frequent category (using df_oh.drop)
# df_oh = df_oh.drop(col_mostFreq, axis=1)
# print(df_oh)


# Rename the columns so that the . used by pandas is replaced with an underscore
# df_oh.columns = col + '_' + df_oh.columns
# print(df_oh)


# Concatenate the onehot data to df_all_onehot (ensure that sort = False !!!!)
# df_all_onehot = pd.concat([df_all_onehot, df_oh], axis = 1, sort = False)


# df_all_onehot.head()

**TODO**

The for loop below will onehot encode all categorical variables and for each, it will drop the most frequent category.  Complete the code below.
 

In [38]:
vars_ind_onehot = []

df_all_onehot = df_all.copy()

for col in vars_ind_categorical:
    print(col)
    
    # use pd.get_dummies on  df_all[col] 
    df_oh = pd.get_dummies(df_all[col], drop_first=False)
    
    # Find the column name of the most frequent category
    col_mostFreq =  df_oh.sum(axis=0).idxmax() 
    
    # Drop the column of the most frequent category
    df_oh = df_oh.drop(col_mostFreq, axis=1, inplace=False)
        
    # Rename the columns to have the original variable name as a prefix
    oh_names = col+'_'+df_oh.columns
    df_oh.columns = oh_names
    
    df_all_onehot = pd.concat([df_all_onehot, df_oh], axis = 1, sort = False)

    del df_all_onehot[col]
    vars_ind_onehot.extend(oh_names)
# df_all_onehot.head()


ms_subclass
ms_zoning
street
alley
lot_shape
land_contour
lot_config
land_slope
neighborhood
condition_1
condition_2
bldg_type
house_style
roof_style
roof_matl
exterior_1st
exterior_2nd
mas_vnr_type
exter_qual
exter_cond
foundation
bsmt_qual
bsmt_cond
bsmt_exposure
bsmtfin_type_1
bsmtfin_type_2
heating
heating_qc
central_air
electrical
kitchen_qual
functional
fireplace_qu
garage_type
garage_finish
garage_qual
garage_cond
paved_drive
fence
misc_feature
sale_type
sale_condition


In [130]:
# Check the dtypes of the resulting data - they should all be numeric
df_all_onehot[vars_ind_onehot].dtypes

ms_subclass_MS_120              uint8
ms_subclass_MS_160              uint8
ms_subclass_MS_180              uint8
ms_subclass_MS_190              uint8
ms_subclass_MS_30               uint8
ms_subclass_MS_45               uint8
ms_subclass_MS_50               uint8
ms_subclass_MS_60               uint8
ms_subclass_MS_70               uint8
ms_subclass_MS_75               uint8
ms_subclass_MS_80               uint8
ms_subclass_MS_85               uint8
ms_subclass_MS_90               uint8
ms_zoning_C (all)               uint8
ms_zoning_FV                    uint8
ms_zoning_RH                    uint8
ms_zoning_RM                    uint8
street_Grvl                     uint8
alley_Grvl                      uint8
alley_Pave                      uint8
lot_shape_IR1                   uint8
lot_shape_IR2                   uint8
lot_shape_IR3                   uint8
land_contour_Bnk                uint8
land_contour_HLS                uint8
land_contour_Low                uint8
lot_config_C

In [131]:
# how many columns on our new data
df_all_onehot.shape

(2922, 220)

alternative is to mark the relevant columns in df_all as dtype factor - we can do that later when we use h2o or lightgbm

In [132]:
# for col in vars_ind_categorical:
#    df_all[col] = df_all[col].astype('category')
# df_all.dtypes #check changes

### Add a fold variable 

to use for selecting train, val and test

TODO

- Create an instance of a random number generator (np.random.RandomState) with a seed of 2018 and call it rng.  
- Use it to generate random integers (rng.randint()) between 0 and 10.  Generate as many numbers as there are rows of df_all 
- save this data to a new column in df_all called 'fold'

In [133]:
# ?np.random.RandomState.randint()
# randint(low, high=None, size=None, dtype='l')


In [ ]:
# np.random.RandomState(seed=None)
rng = np.random.RandomState(seed=2018)
# randint(low, high=None, size=None, dtype='l')
fold = rng.randint(0,11, len(df_all))
df_all['fold'] = fold
df_all_onehot['fold'] = fold
# df_all.head(10)

### Store the dataset and relevant variables

In [137]:
#store = pd.HDFStore(dirPData + '02_df_all.h5')
#store.remove('df_all')
#df_all.to_hdf(store, 'df_all')
#df_all_onehot.to_hdf(store, 'df_all_onehot')
#store.close()

dict_ = {'df_all': df_all,
         'df_all_onehot': df_all_onehot}

f_name = dirPData + '02_df.pickle'

with open(f_name, "wb") as f:
    pickle.dump(dict_, f)
    
del f_name, dict_

In [138]:
dict_ = {'vars_ind_numeric': vars_ind_numeric, 
         'vars_notToUse': vars_notToUse,
         'var_dep': var_dep,
         'vars_ind_categorical': vars_ind_categorical,
         'vars_ind_onehot': vars_ind_onehot
         }


f_name = dirPData + '02_vars.pickle'

with open(f_name, "wb") as f:
    pickle.dump(dict_, f)
    
del f_name, dict_